# Spotify 5000 Songs — Data Overview & Cleaning (Beginner-Friendly)

**Goal:** Understand the dataset, tidy column names, check basic quality (missing values, duplicates), and save a **cleaned CSV** for the modeling notebooks.

**You’ll learn:**
- What each column looks like (types, ranges)
- How to spot missing values and duplicates
- How to quickly inspect feature distributions and correlations
- How to save a clean, reproducible dataset for later steps

> Tip: This notebook is safe to share on GitHub (no secrets), and it writes clean artifacts into `../data/` and `../reports/`.

## 0) Imports & setup

In [ ]:

import pandas as pd
import numpy as np
import re
from pathlib import Path
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (7,5)


## 1) Load CSV and clean column names
Place your raw file at `../data/spotify_5000_songs.csv`. We’ll:
- collapse repeated spaces
- trim whitespace
- keep the **first token** in any oddly-formatted header (works for exports where `name` has extra suffixes)


In [ ]:

DATA = Path("../data/spotify_5000_songs.csv")
assert DATA.exists(), f"Missing data at {DATA}. Place your CSV there."

raw = pd.read_csv(DATA)

def clean_col(c):
    s = re.sub(r"\s+", " ", str(c)).strip()
    return s.split(" ")[0]

df = raw.copy()
df.columns = [clean_col(c) for c in df.columns]

print("Rows, Cols:", df.shape)
df.head(3)


## 2) Column types & quick dictionary
This helps you see which fields are numeric vs. text and what we’ll likely use for modeling.

In [ ]:

df.info()


In [ ]:

# A quick 'data dictionary' starter based on column names
possible_id_cols = [c for c in df.columns if c.lower() in {'id','track_id','uri'} or 'id' in c.lower()]
possible_name_cols = [c for c in df.columns if c.lower() in {'name','song_name','track','title'}]
possible_artist_cols = [c for c in df.columns if 'artist' in c.lower()]
possible_numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

data_dict = pd.DataFrame({
    'column': df.columns,
    'dtype': [df[c].dtype for c in df.columns],
    'example': [df[c].dropna().iloc[0] if df[c].notna().any() else None for c in df.columns],
    'note': [
        'identifier' if c in possible_id_cols else
        'track name' if c in possible_name_cols else
        'artist' if c in possible_artist_cols else
        ('numeric feature' if c in possible_numeric_cols else 'other')
        for c in df.columns
    ]
})
data_dict


## 3) Basic quality checks: missing values & duplicates

In [ ]:

missing = df.isna().sum().sort_values(ascending=False)
missing[missing > 0]


In [ ]:

# Check duplicates by track 'id' if present, else by (name, artist) combo
dup_mask = None
if any(c in df.columns for c in ['id','track_id']):
    key = 'id' if 'id' in df.columns else 'track_id'
    dup_mask = df.duplicated(subset=[key], keep=False)
else:
    name_col = next((c for c in ['name','song_name','title','track'] if c in df.columns), None)
    artist_col = next((c for c in df.columns if 'artist' in c.lower()), None)
    if name_col and artist_col:
        dup_mask = df.duplicated(subset=[name_col, artist_col], keep=False)

if dup_mask is not None:
    duplicates = df[dup_mask].sort_values(list(df.columns)[:3]).head(20)
    print("Duplicate candidates (showing up to 20):")
    display(duplicates)
else:
    print("No suitable key for duplicate detection was found; skipping duplicate listing.")


## 4) Feature distributions (numeric)
We’ll look at summary stats and a few histograms to spot skew/outliers.
Pick 4–6 representative features to keep the notebook fast and readable.

In [ ]:

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
desc = df[num_cols].describe().T
desc[['mean','std','min','25%','50%','75%','max']].round(3)


In [ ]:

to_plot = [c for c in ['energy','valence','tempo','duration_ms','acousticness','instrumentalness','loudness'] if c in num_cols][:6]
for c in to_plot:
    plt.figure()
    df[c].dropna().hist(bins=40)
    plt.title(f'Histogram: {c}')
    plt.xlabel(c); plt.ylabel('count')
    plt.show()


## 5) Correlations (numeric features)
This can hint which features move together (e.g., **energy** and **loudness**).

In [ ]:

corr = df[num_cols].corr(numeric_only=True)
plt.figure(figsize=(8,6))
plt.imshow(corr, cmap='viridis', aspect='auto')
plt.colorbar(label='correlation')
plt.xticks(range(len(num_cols)), num_cols, rotation=90)
plt.yticks(range(len(num_cols)), num_cols)
plt.title('Correlation heatmap (numeric features)')
plt.tight_layout()
plt.show()


**How to read the heatmap**
- Bright (towards 1): features rise/fall together (e.g., energy & loudness)
- Dark (towards -1): features move in opposite directions
- Near 0: little linear relationship

This helps interpret PCA axes and cluster dimensions later.

## 6) Optional quick outlier look (z-score)
We’ll compute simple z-scores and list rows with extreme values for a couple of columns.

In [ ]:

from scipy.stats import zscore

z_df = df[num_cols].apply(lambda s: zscore(s.dropna()), raw=False)
# Align back (zscore drops NaNs, so reindex and keep only finite)
z_df = z_df.reindex(df.index)
extreme_cols = [c for c in ['tempo','duration_ms','loudness'] if c in num_cols]
outlier_rows = {}
for c in extreme_cols:
    if c in z_df.columns:
        z = z_df[c]
        outlier_rows[c] = df[(z.abs() >= 4)].head(10)[[c] + [col for col in ['name','song_name','artist'] if col in df.columns]]
outlier_rows


## 7) Save cleaned artifacts
We’ll write a cleaned CSV and summary tables to reuse across notebooks and for your GitHub repo.

In [ ]:

DATA_OUT = Path("../data/spotify_5000_songs.cleaned.csv")
REPORTS = Path("../reports"); REPORTS.mkdir(parents=True, exist_ok=True)

df.to_csv(DATA_OUT, index=False)
desc_out = REPORTS / "data_summary_numeric.csv"
desc.to_csv(desc_out)

dict_out = REPORTS / "data_dictionary_autogen.csv"
data_dict.to_csv(dict_out, index=False)

print("Saved cleaned CSV:", DATA_OUT)
print("Saved numeric summary:", desc_out)
print("Saved dictionary (auto):", dict_out)


---
## 8) Takeaways & next steps
- Now you have a **cleaned CSV** for consistent runs
- You saw overall distributions and correlations → intuition for PCA and clustering
- If particular features are very skewed, **QuantileTransformer** will likely help

**Continue with:**
- `1_introduction_to_kmeans_Spotify_5000_REWRITE.ipynb` (first K-Means draft)
- `2_scaling_data_Spotify_5000_REWRITE.ipynb` (scaler comparison)
- `3_analysing_k_means__choosing_k_Spotify_5000_REWRITE.ipynb` (choose k)
- `4_PCA_Spotify_5000_REWRITE.ipynb`, `5_DBSCAN_Spotify_5000_REWRITE.ipynb`, `6_AgglomerativeClustering_Spotify_5000_REWRITE.ipynb`